[PyCon KR 2023 - YouTube](https://www.youtube.com/watch?v=dJlX0i-q4ck&list=PLZPhyNeJvHRllQiXsJAryqWmqWrwFxY8I)

In [ ]:
# !pip install -Uq pytube

In [ ]:
import sqlite3
from pytube import Playlist, YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import logging
from datetime import datetime

# 로그 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("pyconkr_subtitle_collection.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# YouTube 재생목록 URL
playlist_url = "https://www.youtube.com/watch?v=dJlX0i-q4ck&list=PLZPhyNeJvHRllQiXsJAryqWmqWrwFxY8I"

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def get_video_info(url):
    try:
        yt = YouTube(url)
        video_id = yt.video_id
        info = {
            'url': url,
            'video_id': video_id,
            'title': yt.title,
            'description': yt.description,
            'views': yt.views,
            'rating': yt.rating,
            'length': yt.length,
            'publish_date': yt.publish_date.isoformat() if yt.publish_date else None,
            'thumbnail_url': yt.thumbnail_url,
            'author': yt.author,
            'channel_id': yt.channel_id,
            'channel_url': yt.channel_url,
        }
        
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
            subtitles = "\n".join([f"[{format_time(entry['start'])}] {entry['text']}" for entry in transcript])
            info['subtitles'] = subtitles
        except Exception as e:
            logging.warning(f"자막을 가져오는 중 오류 발생: {str(e)}")
            info['subtitles'] = "자막 없음"
        
        return info
    except Exception as e:
        logging.error(f"동영상 정보를 가져오는 중 오류 발생: {str(e)}")
        return None

# SQLite 데이터베이스 연결
conn = sqlite3.connect('pyconkr_youtube_playlist.db')
cursor = conn.cursor()

# 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS videos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    url TEXT NOT NULL UNIQUE,
    video_id TEXT,
    title TEXT,
    description TEXT,
    views INTEGER,
    rating FLOAT,
    length INTEGER,
    publish_date TEXT,
    thumbnail_url TEXT,
    author TEXT,
    channel_id TEXT,
    channel_url TEXT,
    subtitles TEXT,
    collected_at TIMESTAMP
)
''')

# YouTube 재생목록에서 모든 URL 가져오기
playlist = Playlist(playlist_url)
video_urls = [video_url for video_url in playlist.video_urls]

# 동영상 정보 수집 및 저장
for url in video_urls:
    cursor.execute('SELECT url FROM videos WHERE url = ?', (url,))
    if cursor.fetchone():
        logging.info(f"동영상 {url}의 정보가 이미 존재합니다. 스킵합니다.")
        continue
    
    info = get_video_info(url)
    if info:
        info['collected_at'] = datetime.now().isoformat()
        placeholders = ', '.join(['?' for _ in info])
        columns = ', '.join(info.keys())
        sql = f"INSERT INTO videos ({columns}) VALUES ({placeholders})"
        cursor.execute(sql, tuple(info.values()))
        logging.info(f"동영상 {url}의 정보를 성공적으로 저장했습니다.")

# 변경 사항 저장
conn.commit()
logging.info("모든 동영상 정보가 'pyconkr_youtube_playlist.db' 데이터베이스에 저장되었습니다.")

# 데이터베이스에서 저장된 정보 확인
logging.info("저장된 동영상 정보를 출력합니다:")
cursor.execute('SELECT * FROM videos')
rows = cursor.fetchall()

for row in rows:
    logging.info(f"URL: {row[1]}")
    logging.info(f"제목: {row[3]}")
    logging.info(f"조회수: {row[5]}")
    logging.info(f"평점: {row[6]}")
    logging.info(f"길이: {row[7]}초")
    logging.info(f"게시일: {row[8]}")
    logging.info(f"채널: {row[11]}")
    logging.info("-" * 80)

# 데이터베이스에서 전체 데이터를 읽어와 판다스 데이터프레임으로 변환 및 출력
df = pd.read_sql_query("SELECT * FROM videos", conn)
print(df)

# 데이터베이스 연결 종료
conn.close()

In [ ]:
df

In [ ]:
df.iloc[-5]["subtitles"][:300]